In [2]:
import networkx as nx
import requests as rq
from routingapp.common.network_stat_utils import get_inter_group_edges

In [3]:
json_graph = rq.get('http://0.0.0.0:8000/graph').json()
graph = nx.json_graph.node_link_graph(json_graph)
inter_group_edges = get_inter_group_edges(graph)
inter_group_edges

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]